# Module 03 — Launch Files & Workspace Organization

## Before you start

This module starts with a problem you will feel personally before we solve it.

---

## 🎥 Watch First

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID"
          style="position:absolute;top:0;left:0;width:100%;height:100%;"
          frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen>
  </iframe>
</div>

---

## Part 1 — Feel the Problem

### 🧪 Experiment 1.1 — Start your full system manually

To run everything you built in Modules 1 and 2, you need to:

**Terminal 1:**
```bash
python3 /home/jovyan/course_materials/scripts/robot_simulator.py
```

**Terminal 2:**
```bash
cd ~/ros2_ws && source install/setup.bash
ros2 run ros2_tutorials py_velocity_publisher
```

**Terminal 3:**
```bash
cd ~/ros2_ws && source install/setup.bash
ros2 run ros2_tutorials py_odom_subscriber
```

**Terminal 4:**
```bash
cd ~/ros2_ws && source install/setup.bash
ros2 run ros2_tutorials py_speed_server
```

Do this now. Actually open all 4 terminals.

**How did that feel?** Imagine doing this every time you test your robot. Imagine 20 nodes instead of 4.

> 📝 Write your reaction.

---

### 💡 This pain is the motivation for Launch Files

> A **Launch File** starts your entire system with a single command.

Kill everything (Ctrl+C in all terminals). We will rebuild this in one command.

---

## Part 2 — Your First Launch File

### Step 1 — Create a bringup package

'Bringup' is the ROS2 convention for the package that starts your robot system:


In [ ]:
%%bash
cd ~/ros2_ws/src
ros2 pkg create ros2_tutorials_bringup --build-type ament_cmake
mkdir -p ros2_tutorials_bringup/launch
echo 'Bringup package created!'


### Step 2 — Write the launch file


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_bringup/launch/robot_system.launch.xml
<?xml version="1.0" encoding="UTF-8"?>
<launch>

  <!-- Robot simulator -->
  <node pkg="ros2_tutorials"
        exec="py_velocity_publisher"
        name="velocity_publisher"/>

  <!-- Odometry monitor -->
  <node pkg="ros2_tutorials"
        exec="py_odom_subscriber"
        name="odom_monitor"/>

  <!-- Speed limit server -->
  <node pkg="ros2_tutorials"
        exec="py_speed_server"
        name="speed_server"/>

</launch>


### Step 3 — Configure CMakeLists.txt


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_bringup/CMakeLists.txt
cmake_minimum_required(VERSION 3.8)
project(ros2_tutorials_bringup)

find_package(ament_cmake REQUIRED)

install(DIRECTORY launch
  DESTINATION share/${PROJECT_NAME}
)

ament_package()


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_bringup/package.xml
<?xml version="1.0"?>
<package format="3">
  <n>ros2_tutorials_bringup</n>
  <version>0.0.1</version>
  <description>Launch files for ROS2 tutorials</description>
  <maintainer email="student@rosforge.com">Student</maintainer>
  <license>Apache-2.0</license>
  <buildtool_depend>ament_cmake</buildtool_depend>
  <exec_depend>ros2_tutorials</exec_depend>
</package>


In [ ]:
%%bash
cd ~/ros2_ws && colcon build && source install/setup.bash
echo 'Build complete!'


### 🧪 Experiment 2.1 — Launch everything with one command

Start the simulator first (it's not a ROS2 package yet so needs a separate terminal):
```bash
python3 /home/jovyan/course_materials/scripts/robot_simulator.py
```

Then in one terminal:
```bash
cd ~/ros2_ws && source install/setup.bash
ros2 launch ros2_tutorials_bringup robot_system.launch.xml
```

**All 3 nodes start simultaneously.** One command. One terminal.

In another terminal, check:
```bash
ros2 node list
```

**How many nodes do you see? Compare this to what you did in Experiment 1.1.**

---

### 🧪 Experiment 2.2 — Launch with parameters

Kill the launch (Ctrl+C). Create a launch file that overrides parameters:


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_bringup/launch/robot_slow.launch.xml
<?xml version="1.0" encoding="UTF-8"?>
<launch>

  <!-- Velocity publisher with slow, tight circle parameters -->
  <node pkg="ros2_tutorials"
        exec="py_velocity_publisher"
        name="velocity_publisher">
    <param name="linear_speed" value="0.1"/>
    <param name="angular_speed" value="2.0"/>
  </node>

  <node pkg="ros2_tutorials"
        exec="py_odom_subscriber"
        name="odom_monitor"/>

</launch>


In [ ]:
%%bash
cd ~/ros2_ws && colcon build && source install/setup.bash
echo 'Done!'


```bash
ros2 launch ros2_tutorials_bringup robot_slow.launch.xml
```

**Watch the robot in RViz.** Same code, different behavior, configured entirely from the launch file.

**Prediction:** If you have 10 different robot configurations (slow, fast, indoor, outdoor), how many Python files do you need to change?

> 📝 Write your answer.

---

## Part 3 — Professional Workspace Organization

### 🧪 Experiment 3.1 — See what you have built

Run this to see your current workspace structure:


In [ ]:
%%bash
find ~/ros2_ws/src -type f -name '*.py' -o -name '*.xml' -o -name '*.srv' | sort


**Can you see the pattern?** You already naturally created:
- A package for nodes (`ros2_tutorials`)
- A package for interfaces (`ros2_tutorials_interfaces`)
- A package for launch files (`ros2_tutorials_bringup`)

This is the **ROS2 standard pattern**. Every professional robot project follows it:

```
ros2_ws/
└── src/
    ├── my_robot_description/     ← URDF, meshes, RViz configs  (ament_cmake)
    ├── my_robot_bringup/         ← Launch files, configs       (ament_cmake)
    ├── my_robot_interfaces/      ← Custom messages/services    (ament_cmake)
    └── my_robot_nodes/           ← Python nodes                (ament_python)
```

One responsibility per package. This makes code reusable, shareable, and maintainable.

---

### 🧪 Experiment 3.2 — Understand build types

You have used two build types. Run:
```bash
cat ~/ros2_ws/src/ros2_tutorials/package.xml | grep buildtool
cat ~/ros2_ws/src/ros2_tutorials_bringup/package.xml | grep buildtool
```

**What's the difference?**

| Package type | Build type | Config file |
|---|---|---|
| Python nodes | `ament_python` | `setup.py` |
| Interfaces / Launch | `ament_cmake` | `CMakeLists.txt` |

**Rule of thumb:** If it contains Python nodes → `ament_python`. Everything else → `ament_cmake`.

---

## 🧠 Conceptual Challenges

**1.** Your system has 8 nodes. You start them all with one launch file. Two crash on startup. How do you find which two without reading all the code?

**2.** You have a launch file that sets `linear_speed: 1.0`. At runtime you run `ros2 param set /velocity_publisher linear_speed 2.0`. You restart the launch file. What speed does the robot use? Why?

**3.** A teammate gives you a ROS2 package with only Python nodes and no launch files. What is the minimum number of things you need to create to run their system reliably?

**4.** You want to create a custom message type `RobotStatus.msg`. In which of the 4 package types does it belong? What build type does it use?

**5.** Your bringup package works perfectly on your machine. A teammate clones the repo and tries to run it. It fails. What is the most likely cause?

---

## 📚 Summary

✅ **Launch files** eliminate multi-terminal chaos — one command starts everything

✅ **Parameters in launch files** mean one codebase, many configurations

✅ **Workspace organization** — description, bringup, interfaces, nodes — each with one job

✅ `ament_python` for nodes, `ament_cmake` for everything else

➡️ **Next:** Module 04 — Visualizing Robots with RViz